In [1]:
from IPython.core.display import HTML
with open ("../../style.css", "r") as file:
    css = file.read()
HTML(css)

# A Shift-Reduce Parser for Arithmetic Expressions

In this notebook we implement a simple *recursive descend* parser for arithmetic expressions.
This parser will implement the following grammar:
$$
  \begin{eqnarray*}
  \mathrm{expr}        & \rightarrow & \mathrm{expr}\;\;\texttt{'+'}\;\;\mathrm{product}   \\
                       & \mid        & \mathrm{expr}\;\;\texttt{'-'}\;\;\mathrm{product}   \\
                       & \mid        & \mathrm{product}                                    \\[0.2cm]
  \mathrm{product}     & \rightarrow & \mathrm{product}\;\;\texttt{'*'}\;\;\mathrm{factor} \\
                       & \mid        & \mathrm{product}\;\;\texttt{'/'}\;\;\mathrm{factor} \\
                       & \mid        & \mathrm{factor}                                     \\[0.2cm]
  \mathrm{factor}      & \rightarrow & \texttt{'('} \;\;\mathrm{expr} \;\;\texttt{')'}     \\
                       & \mid        & \texttt{NUMBER} 
  \end{eqnarray*}
$$

## Implementing a Scanner

In [2]:
import re

The function `tokenize` transforms the string `s` into a list of tokens. See below for an example.

In [3]:
def tokenize(s):
    '''Transform the string s into a list of tokens.  The string s
       is supposed to represent an arithmetic expression.
    '''
    lexSpec = r'''([ \t]+)        |  # blanks and tabs
                  ([1-9][0-9]*|0) |  # number
                  ([()])          |  # parentheses 
                  ([-+*/])        |  # arithmetical operators
                  (.)                # unrecognized character
               '''
    tokenList = re.findall(lexSpec, s, re.VERBOSE)
    result    = []
    for ws, number, parenthesis, operator, error in tokenList:
        if ws:        # skip blanks and tabs
            continue
        elif number:
            result += [ 'NUMBER' ]
        elif parenthesis:
            result += [ parenthesis ]
        elif operator:
            result += [ operator ]
        else:
            result += [ f'ERROR({error})']
    return result

In [4]:
tokenize('1 + 2 * (3 - 4)')

['NUMBER', '+', 'NUMBER', '*', '(', 'NUMBER', '-', 'NUMBER', ')']

In [5]:
class ShiftReduceParser():
    def __init__(self, actionTable, gotoTable):
        self.mActionTable = actionTable
        self.mGotoTable   = gotoTable

In [10]:
def parse(self, TL):
    index   = 0      # points to next token
    Symbols = []     # stack of symbols
    States  = ['s0'] # stack of states, s0 is start state
    TL     += ['$']
    while True:
        q = States[-1]
        t = TL[index]
        print('Symbols:', ' '.join(Symbols + ['|'] + TL[index:]).strip())
        p = self.mActionTable.get((q, t), 'error')
        if p == 'error': 
            return False
        elif p == 'accept':
            return True
        elif p[0] == 'shift':
            s = p[1]
            Symbols += [t]
            States  += [s]
            index   += 1
        elif p[0] == 'reduce':
            head, body = p[1]
            n       = len(body)
            Symbols = Symbols[:-n]
            States  = States [:-n] 
            Symbols = Symbols + [head]
            state   = States[-1]
            States += [ self.mGotoTable[state, head] ]
            
ShiftReduceParser.parse = parse
del parse

In [7]:
%run parse-table.py

## Testing

In [8]:
def test(s): 
    parser = ShiftReduceParser(actionTable, gotoTable)
    TL     = tokenize(s)
    print(f'tokenlist: {TL}\n')
    if parser.parse(TL):
        print('Parse successful!')
    else:
        print('Parse failed!')

In [11]:
test('(1 + 2) * 3')

tokenlist: ['(', 'NUMBER', '+', 'NUMBER', ')', '*', 'NUMBER']

Symbols: | ( NUMBER + NUMBER ) * NUMBER $
Symbols: ( | NUMBER + NUMBER ) * NUMBER $
Symbols: ( NUMBER | + NUMBER ) * NUMBER $
Symbols: ( factor | + NUMBER ) * NUMBER $
Symbols: ( product | + NUMBER ) * NUMBER $
Symbols: ( expr | + NUMBER ) * NUMBER $
Symbols: ( expr + | NUMBER ) * NUMBER $
Symbols: ( expr + NUMBER | ) * NUMBER $
Symbols: ( expr + factor | ) * NUMBER $
Symbols: ( expr + product | ) * NUMBER $
Symbols: ( expr | ) * NUMBER $
Symbols: ( expr ) | * NUMBER $
Symbols: factor | * NUMBER $
Symbols: product | * NUMBER $
Symbols: product * | NUMBER $
Symbols: product * NUMBER | $
Symbols: product * factor | $
Symbols: product | $
Symbols: expr | $
Parse successful!


In [12]:
test('1 * 2 + 3 * (4 - 5) / 2')

tokenlist: ['NUMBER', '*', 'NUMBER', '+', 'NUMBER', '*', '(', 'NUMBER', '-', 'NUMBER', ')', '/', 'NUMBER']

Symbols: | NUMBER * NUMBER + NUMBER * ( NUMBER - NUMBER ) / NUMBER $
Symbols: NUMBER | * NUMBER + NUMBER * ( NUMBER - NUMBER ) / NUMBER $
Symbols: factor | * NUMBER + NUMBER * ( NUMBER - NUMBER ) / NUMBER $
Symbols: product | * NUMBER + NUMBER * ( NUMBER - NUMBER ) / NUMBER $
Symbols: product * | NUMBER + NUMBER * ( NUMBER - NUMBER ) / NUMBER $
Symbols: product * NUMBER | + NUMBER * ( NUMBER - NUMBER ) / NUMBER $
Symbols: product * factor | + NUMBER * ( NUMBER - NUMBER ) / NUMBER $
Symbols: product | + NUMBER * ( NUMBER - NUMBER ) / NUMBER $
Symbols: expr | + NUMBER * ( NUMBER - NUMBER ) / NUMBER $
Symbols: expr + | NUMBER * ( NUMBER - NUMBER ) / NUMBER $
Symbols: expr + NUMBER | * ( NUMBER - NUMBER ) / NUMBER $
Symbols: expr + factor | * ( NUMBER - NUMBER ) / NUMBER $
Symbols: expr + product | * ( NUMBER - NUMBER ) / NUMBER $
Symbols: expr + product * | ( NUMBER - NUMBER ) / NUM

In [13]:
test('11+22*(33-44)/(5-10*5/(4-3))')

tokenlist: ['NUMBER', '+', 'NUMBER', '*', '(', 'NUMBER', '-', 'NUMBER', ')', '/', '(', 'NUMBER', '-', 'NUMBER', '*', 'NUMBER', '/', '(', 'NUMBER', '-', 'NUMBER', ')', ')']

Symbols: | NUMBER + NUMBER * ( NUMBER - NUMBER ) / ( NUMBER - NUMBER * NUMBER / ( NUMBER - NUMBER ) ) $
Symbols: NUMBER | + NUMBER * ( NUMBER - NUMBER ) / ( NUMBER - NUMBER * NUMBER / ( NUMBER - NUMBER ) ) $
Symbols: factor | + NUMBER * ( NUMBER - NUMBER ) / ( NUMBER - NUMBER * NUMBER / ( NUMBER - NUMBER ) ) $
Symbols: product | + NUMBER * ( NUMBER - NUMBER ) / ( NUMBER - NUMBER * NUMBER / ( NUMBER - NUMBER ) ) $
Symbols: expr | + NUMBER * ( NUMBER - NUMBER ) / ( NUMBER - NUMBER * NUMBER / ( NUMBER - NUMBER ) ) $
Symbols: expr + | NUMBER * ( NUMBER - NUMBER ) / ( NUMBER - NUMBER * NUMBER / ( NUMBER - NUMBER ) ) $
Symbols: expr + NUMBER | * ( NUMBER - NUMBER ) / ( NUMBER - NUMBER * NUMBER / ( NUMBER - NUMBER ) ) $
Symbols: expr + factor | * ( NUMBER - NUMBER ) / ( NUMBER - NUMBER * NUMBER / ( NUMBER - NUMBER ) ) $
Sy

In [14]:
test('1+2*3-')

tokenlist: ['NUMBER', '+', 'NUMBER', '*', 'NUMBER', '-']

Symbols: | NUMBER + NUMBER * NUMBER - $
Symbols: NUMBER | + NUMBER * NUMBER - $
Symbols: factor | + NUMBER * NUMBER - $
Symbols: product | + NUMBER * NUMBER - $
Symbols: expr | + NUMBER * NUMBER - $
Symbols: expr + | NUMBER * NUMBER - $
Symbols: expr + NUMBER | * NUMBER - $
Symbols: expr + factor | * NUMBER - $
Symbols: expr + product | * NUMBER - $
Symbols: expr + product * | NUMBER - $
Symbols: expr + product * NUMBER | - $
Symbols: expr + product * factor | - $
Symbols: expr + product | - $
Symbols: expr | - $
Symbols: expr - | $
Parse failed!
